# Transcriptomics Basic Analyses

This notebook demonstrates a standard workflow for analyzing bulk RNA-seq data. The primary goal is to identify differentially expressed genes (DEGs) between different disease states (Crohn's Disease, Ulcerative Colitis, and non-IBD controls) and then determine the biological functions associated with those genes.

**The workflow is divided into four main parts:**
1.  **Data Acquisition & Preprocessing:** Downloading the count matrix and metadata, followed by filtering and normalization.
2.  **Exploratory Data Analysis (QC):** Using Principal Component Analysis (PCA) to visualize sample relationships and identify outliers.
3.  **Differential Expression Analysis (DEA):** Using **PyDESeq2** to find genes that are statistically different between groups.
4.  **Functional Enrichment Analysis:** Using **GOATools** to understand the biological roles of the differentially expressed genes.

Two main libraries used in these analyses PyDESeq2 and GOATools:

- PyDESeq2: This package is a python implementation of the DESeq2 method for differential expression analysis (DEA) with bulk RNA-seq data, originally in R. It aims to facilitate DEA experiments for python users.

- GOATools: A Python library for Gene Ontology analyses

In [ ]:
 ! pip install pydeseq2 goatools

## 1. Data Acquisition and Setup

First, we'll create a structured directory to store our data and results.

We are downloading two essential files from the Inflammatory Bowel Disease Multi'omics Database (IBDMDB):
* **`host_tx_counts.tsv.gz`**: The raw gene count matrix. This table contains the number of reads mapped to each gene (columns) for every sample (rows).
* **`hmp2_metadata_2018-08-20.csv`**: The metadata file. This contains information about each sample, most importantly the `diagnosis` (e.g., CD, UC, nonIBD) which we will use for our comparisons.

In [ ]:
! mkdir transcriptomics
! mkdir transcriptomics/data
! mkdir transcriptomics/results

In [ ]:
! wget https://g-227ca.190ebd.75bc.data.globus.org/ibdmdb/products/HMP2/HTX/host_tx_counts.tsv.gz -O transcriptomics/data/host_tx_counts.tsv.gz
! gunzip transcriptomics/data/host_tx_counts.tsv.gz

In [ ]:
! wget https://g-227ca.190ebd.75bc.data.globus.org/ibdmdb/metadata/hmp2_metadata_2018-08-20.csv -O transcriptomics/data/hmp2_metadata_2018-08-20.csv

## 2. Data Preprocessing and Normalization

This step is crucial for preparing the raw data for analysis.

1.  **Load & Transpose Counts:** The count matrix is loaded and transposed. Most analysis tools (including Scikit-learn's PCA) expect a matrix where **samples are rows** and **genes are columns**.
2.  **Load & Align Metadata:** The metadata is loaded. We then align the count matrix and the metadata to ensure we only analyze samples that exist in *both* files.
3.  **Basic Filtering:** We filter out genes with very low expression. Genes that are not expressed in at least 50% of the samples are removed. This reduces noise and improves the performance of statistical tests.
4.  **Log Transformation:** We apply a `log2(count + 1)` transformation to the count data. This is essential for visualization methods like PCA, as it stabilizes the variance across the range of gene expression levels. It prevents the most highly-expressed genes from dominating the analysis and makes the data more homoscedastic.

In [ ]:
import pandas as pd
import numpy as np


counts_df = pd.read_csv("transcriptomics/data/host_tx_counts.tsv", sep="\t", index_col=0).transpose() # DESeq2 expects samples x genes
metadata_df = pd.read_csv("transcriptomics/data/hmp2_metadata_2018-08-20.csv")
metadata_df = metadata_df.drop_duplicates(subset='External ID', keep='first')
metadata_df = metadata_df.set_index('External ID')
groups = metadata_df[['diagnosis']].reset_index().drop_duplicates().set_index('External ID')

# Keep counts for samples for which we have metadata
counts_df = counts_df.loc[counts_df.index.isin(groups.index), :]
groups = groups.loc[counts_df.index, :]

# Basic Filtering (optional but recommended for low-count genes)
# Keep genes expressed in at least half of the samples
# A better way to filter would be to filter genes that are expressed
# in half of the samples in at least one disease group.n
counts_df = counts_df.loc[:, (counts_df > 0).sum(axis=0) >= counts_df.shape[0]/2]


#print("Normalized Counts Shape:", counts_df.shape)
# 3. Log-transformation
normalized_counts = counts_df.apply(lambda x: x + 1).apply(lambda x: np.log2(x))

print("Normalized Counts Shape:", normalized_counts.shape)

## 3. Exploratory Data Analysis: Principal Component Analysis (PCA)

Before statistical testing, we perform Quality Control (QC) using PCA to visualize the primary sources of variation in our dataset.

* **Method:**
    1.  **StandardScaler:** The log-transformed data is *scaled* (Z-score standardized) to have a mean of 0 and a standard deviation of 1. This ensures all genes contribute equally to the PCA.
    2.  **PCA:** We compute the first two Principal Components (PC1 and PC2), which are the new axes that capture the most variance in the data.
    3.  **Plot:** We plot each sample on these two components and color them by their `diagnosis` group.

* **Interpretation Guidelines:**
    * **Clustering:** Ideally, samples from the same biological group (e.g., `nonIBD`) should cluster together, while different groups should be separate.
    * **Outliers:** Look for samples that are very far from their respective group clusters. These could be potential outliers due to batch effects, sample swaps, or unique biological characteristics.
    * **Variance Explained:** The percentages on the axes (e.g., PC1 (X.XX%) and PC2 (Y.YY%)) show how much of the total variance in the data is captured by that component.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

## Principal Component Analysis (PCA) Plot

# Standardize data (center and scale)
scaler = StandardScaler()
scaled_counts = scaler.fit_transform(normalized_counts)
scaled_df = pd.DataFrame(scaled_counts, index=normalized_counts.index, columns=normalized_counts.columns)

# Perform PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_df)
pca_df = pd.DataFrame(data=principal_components,
                      columns=['PC1', 'PC2'],
                      index=scaled_df.index)

# Add metadata for plotting
pca_df['group'] = [groups.loc[i, 'diagnosis'] for i in pca_df.index]

# Plot PCA
explained_variance = pca.explained_variance_ratio_ * 100

plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='group', data=pca_df,
                palette={'UC': 'blue', 'CD': 'red', 'nonIBD': 'green'}, s=100)
plt.title('PCA of RNA-seq Samples')
plt.xlabel(f'PC1 ({explained_variance[0]:.2f}%)')
plt.ylabel(f'PC2 ({explained_variance[1]:.2f}%)')
plt.grid(True)
plt.show()

## 4. Outlier Identification and Removal

Based on the first PCA plot, several samples appear to be strong outliers, especially along the PC1 axis. This code formalizes their identification and removal to improve the accuracy of our downstream analysis.

* **Method:**
    We use the **1.5 * IQR (Interquartile Range) rule** on the PC1 scores. A sample is flagged as an outlier if its PC1 score is below Q1 - (1.5 * IQR) or above Q3 + (1.5 * IQR).

* **Interpretation:**
    The output lists the sample IDs that are flagged for removal. Removing these samples can significantly improve the power and clarity of the differential expression analysis by reducing unwanted variance.

In [ ]:
# Initialize a boolean series to track outliers
is_outlier_iqr = pd.Series(False, index=pca_df.index)

# Check PC1 scores
Q1_pc1 = pca_df['PC1'].quantile(0.25)
Q3_pc1 = pca_df['PC1'].quantile(0.75)
IQR_pc1 = Q3_pc1 - Q1_pc1

lower_bound_pc1 = Q1_pc1 - 1.5 * IQR_pc1
upper_bound_pc1 = Q3_pc1 + 1.5 * IQR_pc1

# Flag samples outside the bounds
is_outlier_iqr |= (pca_df['PC1'] < lower_bound_pc1) | (pca_df['PC1'] > upper_bound_pc1)

# Get the list of outliers
samples_to_remove_iqr = is_outlier_iqr[is_outlier_iqr].index.tolist()

print("--- Samples Flagged by IQR Rule on PC1")
if samples_to_remove_iqr:
    print(samples_to_remove_iqr)
else:
    print("No samples flagged as outliers by the 1.5 * IQR rule.")

### 4a. Visualizing Outliers

This plot confirms the samples flagged by the IQR rule. The outliers are marked with a distinct diamond shape and annotated with their sample IDs, making them easy to spot.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Add the outlier flag to the main PCA DataFrame for easy plotting
pca_df['is_outlier'] = is_outlier_iqr

# Separate data into inliers and outliers for distinct plotting layers
inliers = pca_df[~pca_df['is_outlier']]
outliers = pca_df[pca_df['is_outlier']]

plt.figure(figsize=(9, 7))

# 1. Plot Inliers (as background, usually smaller and muted)
sns.scatterplot(
    x='PC1',
    y='PC2',
    data=inliers,
    color='silver',
    s=50,
    alpha=0.6
)

# 2. Plot Outliers (highlighted with bold color and size)
if not outliers.empty:
    sns.scatterplot(
        x='PC1',
        y='PC2',
        data=outliers,
        hue='group',
        palette={'UC': 'blue', 'CD': 'red', 'nonIBD': 'green'},
        marker='D', # Use a distinct marker (Diamond)
        s=60,
        edgecolor='black',
        linewidth=1,
        zorder=5, # Ensure they plot on top
        #label='Flagged Outlier (IQR)'
    )

    # Annotate the outliers with their sample IDs
    for sample_id, row in outliers.iterrows():
        plt.annotate(
            sample_id,
            (row['PC1'] + 0.5, row['PC2']), # Offset the text slightly
            fontsize=9,
            color='red',
            weight='bold'
        )

# Final plot configuration
plt.title('PCA Plot Highlighting IQR Outliers', fontsize=16)
plt.xlabel(f'PC1 ({explained_variance[0]:.2f}%)', fontsize=12)
plt.ylabel(f'PC2 ({explained_variance[1]:.2f}%)', fontsize=12)
plt.legend(title='Sample Type', loc='best')
plt.grid(True, linestyle=':', alpha=0.5)
plt.tight_layout()
plt.show()

### 4b. Removing Outliers

Here, we create new dataframes (`filtered_counts_df` and `filtered_metadata`) that exclude the outlier samples identified in the previous step. All subsequent analyses will be performed on this cleaned dataset.

In [ ]:
# To remove them:
filtered_counts_df = counts_df.drop(samples_to_remove_iqr)
filtered_metadata = groups.drop(samples_to_remove_iqr)

### 5. PCA on Filtered Data

After removing the outliers, we re-run the PCA to see if the group separation has improved.

* **Interpretation Guideline:**
    Compare this plot to the first one (in cell 8). The clusters for `CD`, `UC`, and `nonIBD` should appear tighter and more distinct. The variance explained by PC1 and PC2 may also change, as the dominant variance from the outlier samples has been removed.

In [ ]:
# Standardize data (center and scale)
scaler = StandardScaler()
scaled_counts = scaler.fit_transform(filtered_counts_df)
#
scaled_df = pd.DataFrame(scaled_counts, index=filtered_counts_df.index, columns=filtered_counts_df.columns)

# Perform PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_df)
pca_df = pd.DataFrame(data=principal_components,
                      columns=['PC1', 'PC2'],
                      index=scaled_df.index)

# Add metadata for plotting
pca_df['group'] = [groups.loc[i, 'diagnosis'] for i in pca_df.index]

# Plot PCA
explained_variance = pca.explained_variance_ratio_ * 100

plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='group', data=pca_df,
                palette={'UC': 'blue', 'CD': 'red', 'nonIBD': 'green'}, s=100)
plt.title('PCA of RNA-seq Samples')
plt.xlabel(f'PC1 ({explained_variance[0]:.2f}%)')
plt.ylabel(f'PC2 ({explained_variance[1]:.2f}%)')
plt.grid(True)
plt.show()

## 6. Differential Expression Analysis (DEA) with PyDESeq2

This is the core statistical analysis to find genes that are differentially expressed (DE) between disease states. Here, we compare Ulcerative Colitis (UC) vs. Crohn's Disease (CD).

* **Method:**
    `PyDESeq2` is a Python port of the popular R package DESeq2, which is designed for analyzing count data from high-throughput sequencing.
    1.  **`DeseqDataSet`**: We initialize the dataset using the **raw counts** (not the log-transformed data), the cleaned metadata, and the design formula `~ diagnosis`.
    2.  **`dds.deseq2()`**: This single command runs the full pipeline, which includes:
        * Estimating size factors (to normalize for library size differences).
        * Estimating gene-wise dispersions (variance).
        * Fitting a negative binomial model and performing Wald tests to determine significance.
    3.  **`DeseqStats`**: We extract the statistical results for our contrast of interest (`['diagnosis', 'UC', 'CD']`).

* **Interpretation Guideline (Results Table):**
    * **`baseMean`**: The average normalized count for the gene across all samples.
    * **`log2FoldChange`**: The core effect size. A positive value means the gene is **up-regulated in UC** compared to CD. A negative value means it is **down-regulated in UC** (i.e., up-regulated in CD).
    * **`padj`**: The p-value adjusted for multiple testing (using Benjamini-Hochberg, or FDR). **This is the most important value for determining significance.** A `padj` < 0.05 is typically considered significant.
    * **`log10_padj`**: The -log10 transformation of the `padj` value, used for visualization.

In [ ]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

## Differential Expression Analysis (PyDESeq2)
# Create a DeseqDataSet object
dds = DeseqDataSet(
    counts=filtered_counts_df,
    metadata=filtered_metadata,
    design='~ diagnosis'
)

# Run DESeq2 pipeline
dds.deseq2()

# Perform statistical analysis
stat_res = DeseqStats(dds, contrast=['diagnosis', 'UC', 'CD'])
stat_res.summary()
results_df = stat_res.results_df

# Clean up DE table
de_table = results_df.dropna(subset=['padj']).sort_values('padj')
de_table['log10_padj'] = -np.log10(de_table['padj'])

print("--- Differential Expression Table Head ---")
print(de_table.head())

# Save the DE Table
de_table.to_csv('transcriptomics/results/differential_expression_results.csv')

## 7. Visualizing DEA Results: Volcano Plot

A volcano plot is a standard way to visualize a DE analysis. It plots statistical significance (`-log10(padj)`) on the y-axis against the magnitude of change (`log2(FoldChange)`) on the x-axis.

* **Interpretation Guideline:**
    * **Top-Right (Red):** Genes that are **significantly up-regulated** in UC vs. CD (high positive log2FC, low padj).
    * **Top-Left (Blue):** Genes that are **significantly down-regulated** in UC vs. CD (high negative log2FC, low padj).
    * **Center (Grey):** Genes that are *not* significantly differentially expressed.
    * **Dashed Lines:** These represent the thresholds we set for significance (padj < 0.05 and |log2FC| > 1.0).

In [ ]:
## Volcano Plot
plt.figure(figsize=(10, 8))

# Define significance thresholds
lfc_threshold = 1.0 # log2 Fold Change
padj_threshold = 0.05

# Categorize genes for coloring
de_table['color'] = 'grey'
de_table.loc[(de_table['log2FoldChange'].abs() > lfc_threshold) &
             (de_table['padj'] < padj_threshold), 'color'] = 'red'
de_table.loc[(de_table['log2FoldChange'] < -lfc_threshold) &
             (de_table['padj'] < padj_threshold), 'color'] = 'blue'

# Plot
sns.scatterplot(x='log2FoldChange', y='log10_padj', data=de_table,
                hue='color', palette={'grey': 'grey', 'red': 'red', 'blue': 'blue'},
                s=20, alpha=0.6, legend=False)

# Add significance lines
plt.axhline(-np.log10(padj_threshold), color='black', linestyle='--', linewidth=1)
plt.axvline(lfc_threshold, color='black', linestyle='--', linewidth=1)
plt.axvline(-lfc_threshold, color='black', linestyle='--', linewidth=1)

plt.title('Volcano Plot')
plt.xlabel(r'$\log_2$(Fold Change) (Group B vs Group A)')
plt.ylabel(r'$-\log_{10}$(Adjusted P-value)')
plt.show()

## 8. Visualizing DEA Results: MA Plot

An MA plot is another standard visualization that contrasts the log2 Fold Change (M-axis) against the average normalized count (A-axis). It is useful for checking if the differential expression is dependent on the gene's overall expression level.

* **Interpretation Guideline:**
    * **Y-axis:** `log2(FoldChange)`. Points far from the 0-line (black dash) are differentially expressed.
    * **X-axis:** `Mean of Normalized Counts` (on a log scale).
    * **Ideal Plot:** The points should be centered on 0 across the entire range of expression. This plot confirms that our normalization (PyDESeq2 size factors) was successful and there is no strong bias for high- or low-expressed genes.

In [ ]:
## MA Plot
plt.figure(figsize=(10, 8))

# The baseMean column from DESeq2 is the mean of normalized counts
# Plot
sns.scatterplot(x='baseMean', y='log2FoldChange', data=de_table,
                hue='color', palette={'grey': 'grey', 'red': 'red', 'blue': 'blue'},
                s=20, alpha=0.6, legend=False)

# Add horizontal line at M=0
plt.axhline(0, color='black', linestyle='-', linewidth=1)

# Set x-axis to log scale for better visualization of mean expression
plt.xscale('log')

plt.title('MA Plot')
plt.xlabel('Mean of Normalized Counts')
plt.ylabel(r'$\log_2$(Fold Change) (Group B vs Group A)')
plt.show()

## 9. Functional Enrichment Analysis

After identifying *which* genes are significant, the next step is to understand *what* they do. Gene Ontology (GO) Enrichment Analysis helps us find if our list of significant genes is enriched for specific biological processes (BP), molecular functions (MF), or cellular components (CC).

### GOATools Workflow

The following cells perform the full GSEA workflow using the `sig_genes` list we defined in the DEA section (padj < 0.05 and |log2FC| > 1.0).

1.  **File Download:** Downloads the `go-basic.obo` file (the GO hierarchy database) and the `goa_human.gaf` file (the gene-to-GO annotation mapping for humans).
2.  **Annotation Parsing:** The `.gaf` file is read and parsed into a dictionary (`geneid_to_go_dict`) that maps gene symbols (e.g., 'SPSB3') to a set of GO terms (e.g., 'GO:0043161').
3.  **OBO Parsing:** The `.obo` file is loaded into a `GODag` object. This object understands the relationships between GO terms (e.g., 'protein catabolism' *is_a* 'metabolism').
4.  **Enrichment Analysis:**
    * `GOEnrichmentStudy` is initialized with our `pop` (population) of all tested genes and the `assoc` mapping.
    * `run_study_nts()` runs the enrichment test (a Fisher's exact test) on our list of `sig_genes` against the background population.
    * We specify `fdr_bh` (Benjamini-Hochberg) for multiple testing correction.
5.  **Results:** The final table shows the enriched GO terms.

* **Interpretation Guideline (Final Table):**
    * **`term_name`**: The human-readable GO term (e.g., 'extracellular region', 'defense response').
    * **`study_ratio`**: The ratio of genes from our *significant list* associated with this term (e.g., 51 out of 173).
    * **`population_ratio`**: The ratio of genes from our *background* (all tested genes) associated with this term.
    * **`FDR padj`**: The adjusted p-value. **A value < 0.05 is considered significant.**
    * **Conclusion:** The top terms (like 'extracellular region' and 'immune response') suggest that the main differences between UC and CD are related to extracellular proteins and immune system processes, which is biologically expected for inflammatory bowel diseases.

In [ ]:
!wget http://current.geneontology.org/annotations/goa_human.gaf.gz -O transcriptomics/data/goa_human.gaf.gz
!gunzip transcriptomics/data/goa_human.gaf.gz

!wget https://go.princeton.edu/GOTermFinder/goFiles/go-basic.obo -O transcriptomics/data/go-basic.obo

In [ ]:
from goatools.anno.gaf_reader import GafReader
from collections import defaultdict

go_annot = "transcriptomics/data/goa_human.gaf"
ogaf = GafReader(go_annot)

ogaf.associations.pop()

In [ ]:
# The 'associations' attribute is a list of named tuples,
# where each tuple represents a line in the GAF file.

# Convert the list of named tuples to a dictionary
# A common format for GO data is a dictionary where keys are gene IDs
# and values are a set (or list) of GO terms.

geneid_to_go = defaultdict(set)
for assoc in ogaf.associations:
    # 'DB_Object_ID' and 'GO_ID' are typical fields in the named tuples
    gene_id = assoc.DB_Symbol
    go_id = assoc.GO_ID
    geneid_to_go[gene_id].add(go_id)

# Convert defaultdict back to a regular dict if preferred
geneid_to_go_dict = dict(geneid_to_go)

In [ ]:
from goatools.obo_parser import GODag

go_obo = "transcriptomics/data/go-basic.obo"
godag = GODag(go_obo)

In [ ]:
from goatools.go_enrichment import GOEnrichmentStudy

population = filtered_counts_df.columns.tolist()
sig_genes = de_table[(de_table['padj'] < padj_threshold) &
                     (de_table['log2FoldChange'].abs() > lfc_threshold)].index.tolist()

goeaobj = GOEnrichmentStudy(
    pop=filtered_counts_df.columns.tolist(),
    assoc=geneid_to_go_dict,
    obo_dag=godag,
    methods=['fdr_bh'],
    pvalcalc='fisher_scipy_stats')

results = goeaobj.run_study_nts(sig_genes)

In [ ]:
enrichment_results = []
for ntd in sorted(results, key=lambda nt: [nt.p_uncorrected, nt.GO]):
  enrichment_results.append((ntd.NS,
                            ntd.GO,
                            godag.get(ntd.GO).name,
                            '{}/{}'.format(*ntd.ratio_in_study),
                            '{}/{}'.format(*ntd.ratio_in_pop),
                            ntd.p_uncorrected,
                            ntd.p_fdr_bh))
enrichment_results = pd.DataFrame(enrichment_results, columns=['namespace',
                                                               'term_id',
                                                               'term_name',
                                                               'study_ratio',
                                                               'population_ratio',
                                                               'pval_uncorr',
                                                               'FDR padj'
                                                               ]
                                  )

enrichment_results.to_csv('transcriptomics/results/go_enrichment_results.csv')

In [ ]:
enrichment_results.head()